In [1]:
import torch
import torchvision
import os
from glob import glob
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms.functional
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [2]:

class_path = [r"C:\Users\crisp\OneDrive\Skrivebord\eXAM\bikes\bikes", r'C:\Users\crisp\OneDrive\Skrivebord\eXAM\bikes\non-bikes']
class_labels = ["bikes", "non-bikes"]
val_class_path = [r"C:\Users\crisp\OneDrive\Skrivebord\eXAM\validation", r'C:\Users\crisp\OneDrive\Skrivebord\eXAM\val_non_bikes']
val_images, val_labels = [], []
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []
batch_size = 64
num_classes = 2
num_epochs = 120
learning_rate = 0.01

In [3]:
def preprocess(image):
    if image.shape[0] == 1:
        image = image.repeat(3, 1, 1)
    elif image.shape[0] == 4:
        image = image[:3]
    image = image.float() / 255
    image = torchvision.transforms.functional.resize(image, [64, 64], antialias=True)
    return image
train_transforms = transforms.Compose([
    transforms.RandomRotation(85),  # Rotate images by up to 10 degrees
    transforms.Resize((64, 64)),    # Resize images (modify as per your requirement)
    transforms.ToTensor(),          # Convert images to Tensor
    # Add any other transformations like normalization if required
])

images, labels = [], []
extensions = ['jpeg', 'jpg', 'png']

In [4]:
for i, label in enumerate(class_labels):
    filenames = []
    for ext in extensions:
        filenames.extend(glob(os.path.join(class_path[i], f'*.{ext}')))
    
    print(f"Found {len(filenames)} images in class '{label}'.")
    for file in filenames:
        try:
            image = torchvision.io.read_image(file)
            image = preprocess(image)
            images.append(image)
            labels.append(i)  # Label as 0 or 1
            val_images.append(image)
            val_labels.append(i)  # Label as 0 or 1
        except Exception as e:
            print(f"Failed to load image {file}: {e}")

        # except Exception as e:
        #     print(f"Failed to load image {file}: {e}")
if not val_images:
    print("No validation images were loaded. Please check the dataset and file paths.")
else:
    val_images_tensor = torch.stack(val_images).float()
    val_labels_tensor = torch.tensor(val_labels).float()

Found 2027 images in class 'bikes'.
Found 1864 images in class 'non-bikes'.


In [13]:

images_tensor = torch.stack(images).float()
labels_tensor = torch.tensor(labels).float()
val_images_tensor = torch.stack(val_images).float()
val_labels_tensor = torch.tensor(val_labels).float()

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"
train_data = TensorDataset(images_tensor, labels_tensor)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data = TensorDataset(val_images_tensor, val_labels_tensor)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)


In [14]:
# Neural network
net = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Linear(64 * 16 * 16, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 1)
).to(device)

# Loss and optimizer
loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
rotation_transform = transforms.RandomRotation(85)


In [1]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch
    """
    # round predictions to the closest integer (0 or 1)
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()  # convert into float for division
    acc = correct.sum() / len(correct)
    return acc

# Training loop
# Training loop
for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_acc = 0
    for x, y in train_loader:
        # Apply rotation transform to each image in the batch
        x = torch.stack([rotation_transform(img) for img in x])

        x, y = x.to(device), y.to(device)
        # ... rest of your training loop ...
    # Training phase
    net.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        out = net(x)

        loss = loss_function(out.squeeze(), y)
        acc = binary_accuracy(out.squeeze(), y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    # Validation phase
    net.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for x_val, y_val in val_loader:
            x_val, y_val = x_val.to(device), y_val.to(device)
            val_out = net(x_val)
            val_loss += loss_function(val_out.squeeze(), y_val).item()
            val_acc += binary_accuracy(val_out.squeeze(), y_val).item()

    # Calculating average loss and accuracy
    train_loss = epoch_loss / len(train_loader)
    train_acc = epoch_acc / len(train_loader)
    val_loss /= len(val_loader)
    val_acc /= len(val_loader)


    # Save metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    # Printing the results for the current epoch
    
# Plotting

    # Printing the results for the current epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}')

# Save the model after training
torch.save(net.state_dict(), 'bike_Cycle_model.pth')


NameError: name 'num_epochs' is not defined